Jeffrey Dunagin

Homework 9.2

DSC 530

13 February 2022

# 11.1
Suppose one of your co-workers is expecting a baby and you are participating in an office pool to predict the date of birth. Assuming that bets are placed during the 30th week of pregnancy, what variables could you use to make the best prediction? You should limit yourself to variables that are known before the birth, and likely to be available to the people in the pool.

In [2]:
# import data
import first
live, firsts, others = first.MakeFrames()

# only use data for pregnancy length > 30 weeks
live = live[live.prglngth>30]

In [9]:
# join data sets
import chap01soln

resp = chap01soln.ReadFemResp()
resp.index = resp.caseid
join = live.join(resp, on='caseid', rsuffix='_r')

In [ ]:
# find significant variables
import statsmodels.formula.api as smf

t = []
for name in join.columns:
    try:
        if join[name].var() < 1e-7:
            continue
        formula = 'prglngth ~ agepreg + ' + name
        model = smf.ols(formula, data=join)
        if model.nobs < len(join)/2:
            continue
        
        results = model.fit()
    except (ValueError, TypeError):
        continue
    t.append((results.rsquared, name))

In [14]:
# sort
t.sort(reverse=True)
for mse, name in t[:30]:
    print(name, mse)

prglngth 1.0
wksgest 0.8065115675039275
agecon 0.7772278070905594
totalwgt_lb 0.12550330803309884
birthwgt_lb 0.12067945355791987
lbw1 0.10383887474318443
mosgest 0.09562434659774621
prglngth_i 0.0224184207536523
canhaver 0.006072881692014254
datcon01_i 0.005880867019011138
con1mar1_i 0.005607617892541494
nbrnaliv 0.0045801411639411205
mar1con1_i 0.003182814238067544
anynurse 0.0029184839813637398
bfeedwks 0.00256701215878008
rmarout11_i 0.002264199389525934
marout11_i 0.002264199389525934
marcon11_i 0.002264199389525934
pregend1 0.002249407606621956
paydu 0.0021058815474742643
cmlastlb_r 0.002047727311564107
cmlastlb 0.002047727311564107
datend02_i 0.0020339854256564704
datcon02_i 0.0020339854256564704
agecon02_i 0.0020117622161546223
agebaby1 0.0019800046816806915
ageprg02_i 0.0019751611234412048
fmarcon5_i 0.001971805916452851
evuseint 0.0019247289345252216
marcon03_i 0.0018174863473715552


Of the variables that we could possibly know, age of conception (agecon) has the best R^2 value, and therefore is the best predictor. The other good predictors (R^2 > 0.1) are not knowable in normal situations.

# 11.3
If the quantity you want to predict is a count, you can use Poisson regression, which is implemented in StatsModels with a function called `poisson`. It works the same way as `ols` and `logit`. As an exercise, let’s use it to predict how many children a woman has born; in the NSFG dataset, this variable is called `numbabes`.

Suppose you meet a woman who is 35 years old, black, and a college graduate whose annual household income exceeds $75,000. How many children would you predict she has born?

In [3]:
# import data
import nsfg

live = live[live.prglngth>30]
resp = nsfg.ReadFemResp()
resp.index = resp.caseid
join = live.join(resp, on='caseid', rsuffix='_r')

In [12]:
# regression
import statsmodels.formula.api as smf

formula='numbabes ~ age_r + race + totincr + educat'
model = smf.poisson(formula, data=join)
results = model.fit()
results.summary() 

Optimization terminated successfully.
         Current function value: 1.689226
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:               numbabes   No. Observations:                 8884
Model:                        Poisson   Df Residuals:                     8879
Method:                           MLE   Df Model:                            4
Date:                Mon, 07 Feb 2022   Pseudo R-squ.:                 0.02984
Time:                        02:40:37   Log-Likelihood:                -15007.
converged:                       True   LL-Null:                       -15469.
Covariance Type:            nonrobust   LLR p-value:                1.519e-198
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.1545      0.049     23.661      0.000       1.059       1.250
age_r          0.0206      0.001     20.316      0.000       0.019       0.023
race          -0.0802      0.011     -7.043      0.000      -0.102      -0.058
totincr       -0.0198      0.002    -10.547      0.000      -0.023      -0.016
educat        -0.0444      0.003    -15.162      0.000      -0.050      -0.039
==============================================================================
"""

In [13]:
# put in her info
import pandas as pd

columns = ['age_r', 'race', 'totincr', 'educat']
new = pd.DataFrame([[35, 1, 14, 16]], columns=columns)
results.predict(new)

0    2.244372
dtype: float64

We predict her to have 2.24 children, or between 2 and 3.

# 11.4
If the quantity you want to predict is categorical, you can use multinomial logistic regression, which is implemented in StatsModels with a function called `mnlogit`. As an exercise, let’s use it to guess whether a woman is married, cohabitating, widowed, divorced, separated, or never married; in the NSFG dataset, marital status is encoded in a variable called `rmarital`.

Suppose you meet a woman who is 25 years old, white, and a high school graduate whose annual household income is about $45,000. What is the probability that she is married, cohabitating, etc?

In [16]:
formula='rmarital ~ age_r + race + totincr + educat'
model = smf.mnlogit(formula, data=join)
results = model.fit()
results.summary() 

Optimization terminated successfully.
         Current function value: 1.100457
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:               rmarital   No. Observations:                 8884
Model:                        MNLogit   Df Residuals:                     8859
Method:                           MLE   Df Model:                           20
Date:                Mon, 07 Feb 2022   Pseudo R-squ.:                  0.1557
Time:                        02:41:11   Log-Likelihood:                -9776.5
converged:                       True   LL-Null:                       -11579.
Covariance Type:            nonrobust   LLR p-value:                     0.000
==============================================================================
rmarital=2       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.7129      0.305     15.465      0.000       4.116       5.310
age_r         -0.0563      0.006     -9.685      0.000      -0.068      -0.045
race          -0.4868      0.067     -7.270      0.000      -0.618      -0.356
totincr       -0.1384      0.011    -12.098      0.000      -0.161      -0.116
educat        -0.2035      0.019    -10.965      0.000      -0.240      -0.167
------------------------------------------------------------------------------
rmarital=3       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -4.6351      0.971     -4.774      0.000      -6.538      -2.732
age_r          0.1305      0.019      6.863      0.000       0.093       0.168
race          -0.0856      0.169     -0.505      0.614      -0.418       0.246
totincr       -0.3316      0.032    -10.439      0.000      -0.394      -0.269
educat        -0.1015      0.048     -2.114      0.035      -0.196      -0.007
------------------------------------------------------------------------------
rmarital=4       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.3550      0.324     -7.267      0.000      -2.990      -1.720
age_r          0.0640      0.007      9.785      0.000       0.051       0.077
race          -0.3083      0.069     -4.500      0.000      -0.443      -0.174
totincr       -0.2273      0.012    -19.696      0.000      -0.250      -0.205
educat         0.0676      0.017      4.055      0.000       0.035       0.100
------------------------------------------------------------------------------
rmarital=5       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.0714      0.372      2.882      0.004       0.343       1.800
age_r          0.0072      0.007      1.003      0.316      -0.007       0.021
race          -0.5091      0.079     -6.421      0.000      -0.664      -0.354
totincr       -0.3022      0.014    -20.980      0.000      -0.330      -0.274
educat        -0.0127      0.021     -0.599      0.549      -0.054       0.029
------------------------------------------------------------------------------
rmarital=6       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      7.4843      0.296     25.261      0.000       6.904       8.065
age_r         -0.0899      0.006    -15.973      0.000      -0.101      -0.079
race          -1.5675      0.065    -24.037      0.000      -1.695      -1.440
totincr       -0.3014      0.011    -26.346      0.000      -0.324      -0.279
educat        -0.0747      0.017     -4.279      0.000      -0.109      -0.040
==============================================================================
""

In [17]:
columns = ['age_r', 'race', 'totincr', 'educat']
new = pd.DataFrame([[25, 2, 11, 12]], columns=columns)
results.predict(new)

,0,1,2,3,4,5
0,0.712204,0.139107,0.001173,0.033348,0.02781,0.086358


71% chance of being married, 14% chance of not being married but living with opposite sex partner.